In [1]:
import os
import time

from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone

In [2]:
%pwd

'/home/rohwid/GitHub/question-and-answer-chatbot/notebooks'

In [3]:
# Change to the main directory
# So, it's executed from main directory
os.chdir("../")

In [4]:
%pwd

'/home/rohwid/GitHub/question-and-answer-chatbot'

In [5]:
with open('.env') as f:
    os.environ.update(
        line.strip().split('=') for line in f
)

In [6]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_INDEX = os.environ.get('PINECONE_INDEX')

In [7]:
pincone = Pinecone(api_key=PINECONE_API_KEY)
index = pincone.Index(PINECONE_INDEX)

embeddings = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=1536)

query = "What are the specific features or aspects that users appreciate the most in our application?"

docs = []

namespace = "ns-0"

try:
    embedded_query = embeddings.embed_query(query)
    vectors = index.query(
        namespace=namespace,
        vector=embedded_query,
        top_k=5,
        include_values=True
    )
    
    vector_ids = [vector['id'] for vector in vectors['matches']]
    result = index.fetch(ids=vector_ids, namespace=namespace)
except Exception as err:
    raise(err)
else:
    if not vectors:
        raise Exception('No vectors found. Wrong namespace or credentials!')
    for id in vector_ids:
        docs.append(result['vectors'][id]['metadata']['text'])

In [8]:
docs

['google_store_review_comments: Awesome UI, best music app out there!',
 'google_store_review_comments: I enjoy the awesome UI of this app, and it has all the music one can ask for.',
 'google_store_review_comments: Excellent music app',
 'google_store_review_comments: Just started using this app n loving it already  Keep up the great work',
 'google_store_review_comments: Use it every day']